In [1]:
!pip install unidecode ydata-profiling h2o pandasql kmodes apyori efficient-apriori stepmix
!pip install --upgrade mlxtend 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.6 MB/s eta 0:00:00
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=bfad90d98256b8600c1840b1b21a813ad1a17602a2764a98d96153bb6ed20f88
  Stored in directory: /root/.cache/pip/wheels/77/3d/a6/d317a6fb32be58a602b1e8c6b5d6f31f79322da554cad2a5ea
Successfully built apyori
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stepmix 2.2.3 requires scikit-learn<=1.5.0,>=1.0.0, but you have scikit-learn 1.7.2 which i

In [2]:
import joblib
from unidecode import unidecode
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os
import re
import os
from ydata_profiling import ProfileReport
import h2o
from sklearn.cluster import KMeans
from pandasql import sqldf
from kmodes.kmodes import KModes
from mlxtend.frequent_patterns import apriori, association_rules
from efficient_apriori import apriori as ea_apriori
from mlxtend.preprocessing import TransactionEncoder
import re
from collections import defaultdict
import seaborn as sns
from sklearn.model_selection import train_test_split
import copy
from scipy.stats import chi2_contingency
import itertools

In [3]:
with open("/kaggle/input/violencia-intrafamiliar-en-guatemala-ine-2008-2023/label_encoders.pkl", "rb") as f:
    label_encoders = joblib.load(f)


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
df = pd.read_csv("/kaggle/input/violencia-intrafamiliar-en-guatemala-ine-2008-2023/violenciaIntrafamiliarGuatemala2008_2023_v3.csv")

/tmp/ipykernel_13/1360123065.py:1: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/violencia-intrafamiliar-en-guatemala-ine-2008-2023/violenciaIntrafamiliarGuatemala2008_2023_v3.csv")


## Prueba Chi Cuadrada

### Primero hay que eliminar las variables que sí están correlacionadas porque derivan unas de otras

In [5]:
col_list2=['AGR_ALFAB_ENC',
 'AGR_DEDICA_ENC',
 'AGR_ESCOLARIDAD_ENC',
 'AGR_EST_CIV_ENC',
 'AGR_GRUPET_ENC',
 'AGR_NACIONAL_ENC',
 'AGR_OCUP_ENC',
 'AGR_SEXO_ENC',
 'AGR_TRABAJA_ENC',
 'TIPO_DISCAQ_ENC',
 'VIC_ALFAB_ENC',
 'VIC_DEDICA_ENC',
 'VIC_DISC_ENC',
 'VIC_ESCOLARIDAD_ENC',
 'VIC_EST_CIV_ENC',
 'VIC_GRUPET_ENC',
 'VIC_NACIONAL_ENC',
 'VIC_OCUP_ENC',
 'VIC_REL_AGR_ENC',
 'VIC_SEXO_ENC',
 'VIC_TRABAJA_ENC',
 'VIC_OCUP_INE_ENC',
 'AGR_OCUP_INE_ENC',
 'VIC_OCUP_GRUPO_ENC',
 'VIC_OCUP_SUB_GRUPO_PRINCIPAL_ENC',
 'VIC_OCUP_SUB_GRUPO_ENC',
 'VIC_OCUP_GRUPO_UNITARIO_ENC',
 'AGR_OCUP_GRUPO_ENC',
 'AGR_OCUP_SUB_GRUPO_PRINCIPAL_ENC',
 'AGR_OCUP_SUB_GRUPO_ENC',
 'AGR_OCUP_GRUPO_UNITARIO_ENC',
 'CANTIDAD_HIJOS_ENC',
 'CON_HIJ_HOM_ENC',
 'CON_HIJ_MUJ_ENC',
 'VIC_RANGO_EDAD_ENC',
 'AGR_RANGO_EDAD_ENC',
 'VIC_NIV_ESCOLARIDAD_ENC',
 'AGR_NIV_ESCOLARIDAD_ENC',
 'OTROS_AGRESORES_ENC',
 'VIC_OTRAS_VICTIMAS_ENC',
 'VIC_OFICIO_ENC',
 'AGR_OFICIO_ENC',
 'VIOLENCIA_FISICA_ENC',
 'VIOLENCIA_PSICOLOGICA_ENC',
 'VIOLENCIA_SEXUAL_ENC',
 'VIOLENCIA_PATRIMONIAL_ENC',
 'HEC_TIPAGRE_ENC',
 'QUIEN_ES_MAYOR_ENC', 
 'DIF_ALFAB_ENC', 
 'DIF_TIEMPO_REGISTRO_HECHO_ENC', 
 'DIF_SEXO_ENC', 
 'DIF_GRUPET_ENC', 
 'VIC_ES_INDIGENA_ENC', 
 'AGR_ES_INDIGENA_ENC', 
 'DIF_ESCOLARIDAD_AGR_VIC_ENC', 
 'DIF_ESCOLARIDAD_DETALLADA_ENC'
]

### Verificar que las frecuencias esperadas sean mayores que 5

In [6]:
def chi_square_associations(df, cols):
    results = []

    for col1, col2 in itertools.combinations(cols, 2):  # all pairs
        contingency = pd.crosstab(df[col1], df[col2])

        if contingency.shape[0] > 1 and contingency.shape[1] > 1:
            chi2, p, dof, expected = chi2_contingency(contingency)

            # Validation checks
            expected_flat = expected.flatten()
            valid_cells = (expected_flat >= 5).sum() # Counts the amount of cells that their expected vals are bigger or equal than 5 in a np array [True, False...]
            total_cells = expected_flat.size # size of the np array
            min_expected = expected_flat.min() # min value of expected should be bigger than 0
            valid_ratio = valid_cells / total_cells # min valid ratio of 80%
            is_valid = (min_expected > 1) and (valid_ratio >= 0.8)

            results.append({
                "var1": col1,
                "var2": col2,
                "chi2": chi2,
                "dof": dof,
                "p_value": p,
                "valid": is_valid,
                "min_expected": min_expected,
                "valid_ratio": round(valid_ratio, 2)
            })

    results_df = pd.DataFrame(results)
    return results_df.sort_values("p_value")

# Example usage:
results_df = chi_square_associations(df, col_list2)

# Filter only valid and significant results
significant_valid = results_df[(results_df["valid"]) & (results_df["p_value"] < 0.05)]

print(significant_valid.head(20))


                    var1                         var2          chi2  dof  \
0          AGR_ALFAB_ENC               AGR_DEDICA_ENC   5984.913784   12   
936      VIC_TRABAJA_ENC           QUIEN_ES_MAYOR_ENC  17700.212762    6   
934      VIC_TRABAJA_ENC    VIOLENCIA_PATRIMONIAL_ENC   2002.927596    2   
932      VIC_TRABAJA_ENC    VIOLENCIA_PSICOLOGICA_ENC   2382.149843    2   
931      VIC_TRABAJA_ENC         VIOLENCIA_FISICA_ENC   8577.578327    2   
930      VIC_TRABAJA_ENC               AGR_OFICIO_ENC  30365.640960   30   
926      VIC_TRABAJA_ENC      AGR_NIV_ESCOLARIDAD_ENC  26338.428258   10   
925      VIC_TRABAJA_ENC      VIC_NIV_ESCOLARIDAD_ENC  80596.708899   10   
924      VIC_TRABAJA_ENC           AGR_RANGO_EDAD_ENC   4380.720533   10   
923      VIC_TRABAJA_ENC           VIC_RANGO_EDAD_ENC  54134.744028   10   
922      VIC_TRABAJA_ENC              CON_HIJ_MUJ_ENC   5173.527496    4   
921      VIC_TRABAJA_ENC              CON_HIJ_HOM_ENC   5183.092120    4   
920      VIC

In [7]:
print(results_df.shape, significant_valid.shape) # no todas las comparaciones entre variables son validas

(1540, 8) (831, 8)


In [8]:
list_of_tuples = list(significant_valid[['var1', 'var2']].drop_duplicates().itertuples(index=False, name=None))

Por lo tanto, se concluye que no todas las reglas apriori serán válidas dado el test de chi cuadrado con las condiciones de un valor esperado de 5 al menos en el 80% de las celdas de la tabla de contingencia y que todos los valores de las celdas posean un valor esperado mayor que 1. Por lo que de 1509 posibles cruces de variables, únicamente 1089 serán aceptables.

## Apriori

### Dividir el dataset

In [9]:
df = df[(df['HEC_ANO']>=2008)&(df['HEC_ANO']<=2023)]

In [10]:
df['VIC_RANGO_EDAD'].value_counts()

VIC_RANGO_EDAD
adulto joven    409353
adulto medio    104617
adolescente      32483
adulto mayor     24812
ignorado          7716
ninio             3913
Name: count, dtype: int64

In [11]:
df_menores = df[(df['VIC_RANGO_EDAD']=='ninio')|(df['VIC_RANGO_EDAD']=='adolescente')]

In [12]:
df_anciano = df[df['VIC_RANGO_EDAD']=='adulto mayor']

In [13]:
df_mujeres = df[((df['VIC_RANGO_EDAD']=='adulto joven')|(df['VIC_RANGO_EDAD']=='adulto medio')) & (df['VIC_SEXO']=='mujeres')]

In [14]:
df_hombres = df[((df['VIC_RANGO_EDAD']=='adulto joven')|(df['VIC_RANGO_EDAD']=='adulto medio')) & (df['VIC_SEXO']=='hombres')]

In [15]:
df_mujeres.shape

(462046, 190)

In [16]:
df_hombres.shape

(51924, 190)

In [17]:
df_anciano.shape

(24812, 190)

In [18]:
df_menores.shape

(36396, 190)

In [19]:
df_anciano['HEC_ANO'].value_counts()

HEC_ANO
2023.0    2328
2022.0    2206
2021.0    2037
2011.0    1703
2010.0    1675
2019.0    1672
2009.0    1660
2018.0    1472
2020.0    1443
2017.0    1350
2015.0    1296
2016.0    1291
2014.0    1288
2013.0    1281
2012.0    1267
2008.0     843
Name: count, dtype: int64

In [20]:
df_menores['HEC_ANO'].value_counts()

HEC_ANO
2010.0    3467
2009.0    3056
2011.0    2998
2013.0    2833
2012.0    2773
2014.0    2743
2015.0    2365
2016.0    2202
2008.0    2095
2017.0    1934
2018.0    1898
2019.0    1736
2023.0    1709
2021.0    1571
2020.0    1519
2022.0    1497
Name: count, dtype: int64

In [21]:
df_hombres['HEC_ANO'].value_counts()

HEC_ANO
2022.0    4527
2023.0    4422
2021.0    4140
2019.0    3685
2009.0    3570
2018.0    3227
2020.0    3172
2017.0    3155
2010.0    3116
2011.0    3001
2013.0    2825
2016.0    2824
2014.0    2731
2015.0    2719
2012.0    2576
2008.0    2234
Name: count, dtype: int64

In [22]:
df_mujeres['HEC_ANO'].value_counts()

HEC_ANO
2010.0    36699
2009.0    35588
2011.0    33993
2013.0    31396
2012.0    31346
2022.0    29750
2021.0    29701
2014.0    29038
2023.0    28784
2015.0    26931
2019.0    26327
2016.0    26291
2018.0    25177
2017.0    25176
2020.0    23372
2008.0    22477
Name: count, dtype: int64

In [23]:
df_mujeres['VIC_ES_INDIGENA'].value_counts()

VIC_ES_INDIGENA
no          263383
si          143226
ignorado     55437
Name: count, dtype: int64

In [24]:
df_mujeres.columns[100:]

Index(['DIF_GRUPET', 'VIC_ES_INDIGENA', 'AGR_ES_INDIGENA',
       'DIF_ESCOLARIDAD_AGR_VIC', 'DIF_ESCOLARIDAD_DETALLADA', 'AGR_ALFAB_ENC',
       'AGR_DEDICA_ENC', 'AGR_ESCOLARIDAD_ENC', 'AGR_EST_CIV_ENC',
       'AGR_GRUPET_ENC', 'AGR_NACIONAL_ENC', 'AGR_OCUP_ENC', 'AGR_SEXO_ENC',
       'AGR_TRABAJA_ENC', 'DEPTO_MCPIO_ENC', 'HEC_AREA_ENC',
       'HEC_DEPTOMCPIO_ENC', 'HEC_MES_ENC', 'HEC_RECUR_DENUN_ENC',
       'HEC_TIPAGRE_ENC', 'INST_DENUN_HECHO_ENC', 'INST_DONDE_DENUNCIO_ENC',
       'MES_EMISION_ENC', 'QUIEN_REPORTA_ENC', 'TIPO_DISCAQ_ENC',
       'VIC_ALFAB_ENC', 'VIC_DEDICA_ENC', 'VIC_DISC_ENC',
       'VIC_ESCOLARIDAD_ENC', 'VIC_EST_CIV_ENC', 'VIC_GRUPET_ENC',
       'VIC_NACIONAL_ENC', 'VIC_OCUP_ENC', 'VIC_REL_AGR_ENC', 'VIC_SEXO_ENC',
       'VIC_TRABAJA_ENC', 'DEPTO_ENC', 'HEC_DEPTO_ENC', 'CONDUCENTE_ENC',
       'MEDIDAS_SEGURIDAD_ENC', 'LEY_APLICABLE_ENC',
       'ORGANISMO_JURISDICCIONAL_ENC', 'TIPO_MEDIDA_ENC',
       'ORGANISMO_REMITE_ENC', 'ARTICULOVIF_ENC', 'ARTICUL

### Separar los datos en train y test

In [25]:
def split_data(df):
    X_train, X_test = train_test_split(df, test_size=0.3, random_state=42) 
    return X_train, X_test
def split_train_test(df):
    train_test = {}
    for year in range(2008, 2023):
        temp_df = df[df['HEC_ANO']==year]
        train_test[year] = split_data(temp_df)
        # calculate_kmodes(temp_df, year)
    return train_test

In [26]:
sets = {}
tipos = ['menores', 'ancianos', 'hombres', 'mujeres']
for index, dataframe in enumerate([df_menores, df_anciano, df_hombres, df_mujeres]):
    sets[tipos[index]] = split_train_test(dataframe)

In [27]:
reglas = copy.deepcopy(sets)

### Generar asociaciones

In [28]:
def get_if_then(rule):
    if_p, then_p = str(rule).split('->')
    if_p = re.sub(r'[{} ]', '', if_p).split(',')
    then_p = re.sub(r'[{}]', '', then_p).split(',')
    return tuple(if_p), then_p  # tuple for hashable dict key

def extract_rules_from_2(rules):
    return [rule for rule in rules if len(rule) == 2]

def select_rules_for_item(rules):
    rules_dict = defaultdict(list)
    for rule in extract_rules_from_2(rules):
        if_p, _ = get_if_then(rule)
        rules_dict[if_p].append(rule)
    return dict(rules_dict)

def order_rules(rules):
    return sorted(rules, key=lambda r: (-r.conviction, -r.support))

def order_all_rules(rules_dict):
    return {key: order_rules(rules) for key, rules in rules_dict.items()}


In [29]:
def generar_transacciones(df):
    """Convierte un DataFrame en lista de transacciones estilo Apriori."""
    return df.astype(str).apply(
        lambda row: [f"{col}={val}" for col, val in row.items()],
        axis=1
    ).tolist()

def generar_reglas_globales(transacciones, min_support=0.3, min_confidence=0.8, max_length=2):
    """Ejecuta Apriori una sola vez y devuelve todas las reglas frecuentes."""
    return ea_apriori(
        transacciones,
        min_support=min_support,
        min_confidence=min_confidence,
        max_length=max_length,
        output_transaction_ids = True
    )

def filtrar_reglas_por_columnas(reglas, cols):
    """Filtra reglas que incluyan todas las columnas dadas."""
    return [
        r for r in reglas
        if any(c in item for item in set(r.lhs).union(r.rhs) for c in cols)
    ]

def generar_relaciones_por_pares(df, min_support=0.3, min_confidence=0.8):
    """Genera relaciones para cada par de columnas de un DataFrame."""
    # 1. Precompute transactions
    transacciones = generar_transacciones(df)

    # 2. Run Apriori once
    itemsets, reglas_globales = generar_reglas_globales(
        transacciones,
        min_support=min_support,
        min_confidence=min_confidence,
        max_length=2
    )
    return itemsets, reglas_globales
def filter_valid_rules(rules): # Esto para filtrar aquellas reglas que si se pueden comparar por chi cuadrada
    return [rule for rule in rules if (rule.lhs[0].split("=")[0],rule.rhs[0].split("=")[0]) in list_of_tuples]

In [30]:
VARIABLES = ['AGR_ALFAB_ENC',
 'AGR_DEDICA_ENC',
 'AGR_ESCOLARIDAD_ENC',
 'AGR_EST_CIV_ENC',
 'AGR_GRUPET_ENC',
 'AGR_NACIONAL_ENC',
 'AGR_OCUP_ENC',
 'AGR_SEXO_ENC',
 'AGR_TRABAJA_ENC',
 'TIPO_DISCAQ_ENC',
 'VIC_ALFAB_ENC',
 'VIC_DEDICA_ENC',
 'VIC_DISC_ENC',
 'VIC_ESCOLARIDAD_ENC',
 'VIC_EST_CIV_ENC',
 'VIC_GRUPET_ENC',
 'VIC_NACIONAL_ENC',
 'VIC_OCUP_ENC',
 'VIC_REL_AGR_ENC',
 'VIC_SEXO_ENC',
 'VIC_TRABAJA_ENC',
 'VIC_OCUP_INE_ENC',
 'AGR_OCUP_INE_ENC',
 'VIC_OCUP_GRUPO_ENC',
 'VIC_OCUP_SUB_GRUPO_PRINCIPAL_ENC',
 'VIC_OCUP_SUB_GRUPO_ENC',
 'VIC_OCUP_GRUPO_UNITARIO_ENC',
 'AGR_OCUP_GRUPO_ENC',
 'AGR_OCUP_SUB_GRUPO_PRINCIPAL_ENC',
 'AGR_OCUP_SUB_GRUPO_ENC',
 'AGR_OCUP_GRUPO_UNITARIO_ENC',
 'CANTIDAD_HIJOS_ENC',
 'CON_HIJ_HOM_ENC',
 'CON_HIJ_MUJ_ENC',
 'VIC_RANGO_EDAD_ENC',
 'AGR_RANGO_EDAD_ENC',
 'VIC_NIV_ESCOLARIDAD_ENC',
 'AGR_NIV_ESCOLARIDAD_ENC',
 'OTROS_AGRESORES_ENC',
 'VIC_OTRAS_VICTIMAS_ENC',
 'VIC_OFICIO_ENC',
 'AGR_OFICIO_ENC',
 'VIOLENCIA_FISICA_ENC',
 'VIOLENCIA_PSICOLOGICA_ENC',
 'VIOLENCIA_SEXUAL_ENC',
 'VIOLENCIA_PATRIMONIAL_ENC',
 'HEC_TIPAGRE_ENC',
 'QUIEN_ES_MAYOR_ENC', 
 'DIF_ALFAB_ENC', 
 'DIF_TIEMPO_REGISTRO_HECHO_ENC', 
 'DIF_SEXO_ENC', 
 'VIC_ES_INDIGENA_ENC', 
 'AGR_ES_INDIGENA_ENC', 
 'DIF_ESCOLARIDAD_AGR_VIC_ENC', 
 'DIF_ESCOLARIDAD_DETALLADA_ENC'
]
itemsets = {}
sorted_rules = {}
for division, value in sets.items():
    itemsets[division] = {}
    sorted_rules[division] = {}
    for year, dataframe in value.items():
        train_dataset = dataframe[0]
        # Generar las relaciones por anio
        itemsets[division][year], reglas[division][year] = generar_relaciones_por_pares(train_dataset[VARIABLES], min_support=0.3, min_confidence=0.6)
        reglas[division][year] = filter_valid_rules(reglas[division][year])
        # Ordenar las reglas por caracteristica
        sorted_rules[division][year] = order_all_rules(select_rules_for_item(extract_rules_from_2(reglas[division][year])))

In [31]:
def count_repeated_rules(data):
    all_keys = sorted(set().union(*data.values()))
    
    # Crear el DataFrame
    df = pd.DataFrame([
        {key: key in year_keys for key in all_keys}
        for year, year_keys in data.items()
    ], index=data.keys())
    
    # Convertir booleanos a enteros si lo prefieres
    return df.astype(int)
for key in sorted_rules.keys():
    data = sorted_rules[key]
    print(count_repeated_rules(data).sum().sort_values(ascending=False))

(AGR_ALFAB_ENC=0,)                        15
(CON_HIJ_HOM_ENC=2,)                      15
(DIF_ALFAB_ENC=2,)                        15
(HEC_TIPAGRE_ENC=2,)                      15
(AGR_DEDICA_ENC=3,)                       15
(QUIEN_ES_MAYOR_ENC=0,)                   15
(TIPO_DISCAQ_ENC=3,)                      15
(VIC_ALFAB_ENC=0,)                        15
(VIC_DEDICA_ENC=5,)                       15
(VIC_DISC_ENC=2,)                         15
(VIC_ES_INDIGENA_ENC=1,)                  15
(VIC_GRUPET_ENC=2,)                       15
(VIC_NACIONAL_ENC=1,)                     15
(VIC_NIV_ESCOLARIDAD_ENC=4,)              15
(VIC_OCUP_GRUPO_ENC=1,)                   15
(VIC_OFICIO_ENC=14,)                      15
(VIC_RANGO_EDAD_ENC=0,)                   15
(VIC_REL_AGR_ENC=0,)                      15
(VIC_SEXO_ENC=1,)                         15
(VIC_TRABAJA_ENC=1,)                      15
(VIOLENCIA_FISICA_ENC=1,)                 15
(VIOLENCIA_PATRIMONIAL_ENC=0,)            15
(VIOLENCIA

### Filtrar aquellas reglas que no cumplan los umbrales

- Lift > 1.0
- Soporte > 0.3
- Convicción > 1.2

In [32]:
def filter_rules(rules):
    return [rule for rule in rules if rule.conviction>1.2 and rule.lift>1.0 and rule.support >= 0.3]

In [33]:
filtered_rules = {}
for division, value in sorted_rules.items():
    filtered_rules[division] = {}
    for year, rules in value.items():
        # Ordenar las reglas por caracteristica
        filtered_rules[division][year] = {key: filter_rules(rules) for key, rules in rules.items() }

### Desencodear las reglas para su posterior interpretación

In [34]:
for div in filtered_rules.keys():
    for x in range(2008,2023):
        print(f"Cantidad de reglas validas encontradas por division - {div} y anio - {x}:",len(filtered_rules[div][x].keys()))

Cantidad de reglas validas encontradas por division - menores y anio - 2008: 44
Cantidad de reglas validas encontradas por division - menores y anio - 2009: 45
Cantidad de reglas validas encontradas por division - menores y anio - 2010: 45
Cantidad de reglas validas encontradas por division - menores y anio - 2011: 48
Cantidad de reglas validas encontradas por division - menores y anio - 2012: 47
Cantidad de reglas validas encontradas por division - menores y anio - 2013: 43
Cantidad de reglas validas encontradas por division - menores y anio - 2014: 45
Cantidad de reglas validas encontradas por division - menores y anio - 2015: 44
Cantidad de reglas validas encontradas por division - menores y anio - 2016: 42
Cantidad de reglas validas encontradas por division - menores y anio - 2017: 46
Cantidad de reglas validas encontradas por division - menores y anio - 2018: 43
Cantidad de reglas validas encontradas por division - menores y anio - 2019: 48
Cantidad de reglas validas encontradas p

In [35]:
def extract_rule(rule):
    antecedent_variable, antecedent_val = rule.lhs[0].split("=")
    consequent_variable, consequent_val = rule.rhs[0].split("=")

    # Remove all occurrences of '_ENC' from variable names
    antecedent_variable = antecedent_variable.replace('_ENC', '')
    consequent_variable = consequent_variable.replace('_ENC', '')

    return antecedent_variable, antecedent_val, consequent_variable, consequent_val


### Interpretación de reglas

In [36]:
amount_of_rules = 0
for div in filtered_rules.keys():
    for x in range(2008,2009):
        for variable in filtered_rules[div][x]:
            var = variable[0].replace("_ENC","")
            print(f"____VARIABLE___{var}")
            for rule in filtered_rules[div][x][variable]:
                a_v, a_vl, c_v, c_vl = extract_rule(rule)
                avl1 = label_encoders[a_v].inverse_transform([int(a_vl)])[0]
                avl2 = label_encoders[c_v].inverse_transform([int(c_vl)])[0]
                print(f"    {a_v}:{avl1}=>{c_v}:{avl2} - lift {rule.lift} - conviction {rule.conviction} - support {rule.support}")
                amount_of_rules+=1
print(f"Cantidad de reglas:{amount_of_rules}")

____VARIABLE___AGR_ALFAB=0
    AGR_ALFAB:alfabeta=>DIF_ALFAB:iguales - lift 1.119106319804896 - conviction 1.699589841231346 - support 0.703956343792633
    AGR_ALFAB:alfabeta=>VIC_ALFAB:alfabeta - lift 1.0746823358261544 - conviction 1.4567912924840103 - support 0.703956343792633
    AGR_ALFAB:alfabeta=>QUIEN_ES_MAYOR:agresor - lift 1.0322089713825788 - conviction 1.3435583506144555 - support 0.7435197817189632
    AGR_ALFAB:alfabeta=>AGR_NIV_ESCOLARIDAD:primaria - lift 1.2329688814129522 - conviction 1.3136477497554022 - support 0.5061391541609823
____VARIABLE___AGR_DEDICA=3
    AGR_DEDICA:ignorado=>OTROS_AGRESORES:ignorado - lift 1.0006825938566553 - conviction 682128.2401091588 - support 0.8635743519781719
    AGR_DEDICA:ignorado=>AGR_TRABAJA:si - lift 1.1579778830963665 - conviction 2.8041908333886054 - support 0.8028649386084584
    AGR_DEDICA:ignorado=>AGR_SEXO:hombres - lift 1.075973223670865 - conviction 2.1709299299951628 - support 0.8144611186903138
    AGR_DEDICA:ignorado=>

## Desarrollo de perfiles

### Kmodes

In [37]:
km = KModes(n_clusters=4, init='Huang', n_init=5, verbose=1)

In [38]:
# clusters = km.fit_predict(
#     df[[]],
#     init='Huang',
#     n_init = 10,
#     verbose = 1
# )

### Latent Class Analysis